In [1]:
import tensorflow as tf

In [ ]:
def fit(self):
    self.graph = tf.Graph()
    with self.graph.as_default():
        self.X = tf.placeholder(self.dtype, shape=self.data.shape)
        
        # Perform SVD
        singular_values, u, _ = tf.svd(self.X)
        
        # Create sigma matrix
        sigma = tf.diag(singular_values)
        
    with tf.Session(graph=self.graph) as session:
        self.u, self.singular_values, self.sigma = sess.run([u, singular_values, sigma], feed_dict={self.X: self.data})